In [1]:
# Install dependencies 
%pip install langchain-groq langchain langchain-community

from langchain_groq import ChatGroq
from langchain.agents import initialize_agent, Tool, AgentType
from langchain.memory import ConversationBufferMemory
from dotenv import load_dotenv
import os


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


In [2]:
# Set GROQ API key
load_dotenv()
os.environ["GROQ_API_KEY"] = os.getenv('groq-apikey')

In [3]:
# Use Groq llama3-8b-8192 as the LLM
llm = ChatGroq(model_name="llama3-8b-8192", temperature=0.7)

In [15]:
# Mock knowledge base for paragraph themes and examples
PARAGRAPH_DATABASE = {
    "climate_change": "Climate change is one of the most pressing global challenges, marked by rising temperatures, melting glaciers, and shifting weather patterns. Human activities like burning fossil fuels contribute significantly to this crisis.",
    "technology_in_education": "Technology has revolutionized education by providing digital tools that enhance learning experiences, offer flexibility, and increase accessibility for students around the globe.",
    "importance_of_sports": "Sports play a vital role in maintaining physical fitness, building teamwork skills, and promoting discipline and perseverance among individuals.",
    "internet_addiction": "The internet, while beneficial, has also led to addiction in many users, affecting mental health, productivity, and real-life social interactions.",
    "benefits_of_reading": "Reading is a habit that strengthens the brain, improves concentration, and opens up a world of imagination and knowledge to the reader.",
    "plastic_pollution": "Plastic pollution poses a severe threat to marine life and ecosystems, with millions of tons of plastic waste entering our oceans each year.",
    "space_exploration": "Space exploration expands our understanding of the universe, drives technological innovation, and inspires future generations to dream big.",
    "healthy_lifestyle": "Living a healthy lifestyle involves balanced nutrition, regular physical activity, sufficient sleep, and managing stress effectively.",
    "digital_privacy": "Digital privacy is a growing concern in the modern age, where personal data is often tracked and used without users' explicit consent.",
    "artificial_intelligence": "Artificial intelligence is transforming industries by automating tasks, improving decision-making, and enabling smarter technology applications."
}


# Tool functions
def paragraph_writer(prompt: str) -> str:
    for title, content in PARAGRAPH_DATABASE.items():
        if title.replace("_", " ") in prompt.lower():
            return f"Auto-detected topic:\n\n{content}"
    return "No matching paragraph found, but I can suggest something else!"

# Word count function
def count_words(paragraph: str) -> str:
    word_count = len(paragraph.split())
    return f"The paragraph contains {word_count} words."


# Define tools
tools = [
    Tool(
        name="Paragraph_Writer",
        func=paragraph_writer,
        description="Use for writing educational or creative paragraphs or retrieving topic-based content."
    ),
    Tool(
        name="Word_Count",
        func=count_words,
        description="Use for counting the number of words in a given paragraph."
    )
]

In [5]:
# Add memory
memory = ConversationBufferMemory(memory_key="chat_history")

# Initialize the agent
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    memory=memory,
    verbose=True,
    handle_parsing_errors=True
)

C:\Users\shahs_snohmej\AppData\Local\Temp\ipykernel_33372\193109754.py:5: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.3.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  agent = initialize_agent(


In [16]:
# Few-shot example prompt builder for paragraph writer
def build_paragraph_writer_prompt(user_prompt: str) -> str:
    few_shot_examples = """
        Prompt: Write a paragraph on climate change
        Response: Climate change is one of the most pressing global challenges, marked by rising temperatures, melting glaciers, and shifting weather patterns. Human activities like burning fossil fuels contribute significantly to this crisis.

        Prompt: Write a paragraph on the benefits of reading
        Response: Reading is a habit that strengthens the brain, improves concentration, and opens up a world of imagination and knowledge to the reader.

        Prompt: Write a paragraph on internet addiction
        Response: Internet addiction has become a growing concern in the digital age. Prolonged screen time can disrupt sleep, reduce real-world social interactions, and negatively affect mental health, especially in younger populations.
    """

    user_example = f"\nPrompt: {user_prompt}\nResponse: "
    return few_shot_examples + user_example

In [18]:
### Example interaction-1
query = "Write a paragraph on plastic pollution"
prmpt = build_paragraph_writer_prompt(query)
response = agent.run(query)
print(f"Q: {query}\nResponse:\n{response}\n")

query = f"What is the word count of this paragraph: {response}?"
response = agent.run(query)
print(f"Q: {query}\nResponse:\n{response}")



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? No
AI: Plastic pollution is a pressing global issue that demands immediate attention and collective action. The staggering amount of plastic waste that enters our oceans every year is having a devastating impact on marine life, from entangling and suffocating animals to contaminating their habitats with toxic chemicals. Moreover, plastic pollution has severe consequences for human health, as microplastics have been found in our food and water supply. It's crucial that we adopt sustainable practices, reduce our plastic usage, increase our recycling efforts, and develop innovative solutions to mitigate the effects of plastic pollution. By working together, we can create a healthier, more sustainable future for our planet and ensure the well-being of future generations.

> Finished chain.
Q: Write a paragraph on plastic pollution
Response:
Plastic pollution is a pressing global issue that demands immediate attention

In [19]:
### Example interaction-2
query = "Write a paragraph on climate change"
prmpt = build_paragraph_writer_prompt(query)
response = agent.run(query)
print(f"Q: {query}\nResponse:\n{response}\n")

query = f"What is the word count of this paragraph: {response}?"
response = agent.run(query)
print(f"Q: {query}\nResponse:\n{response}")



> Entering new AgentExecutor chain...
Could not parse LLM output: `I'd be happy to assist with that!

Here's a paragraph on climate change:

Climate change is one of the most pressing issues of our time, with far-reaching consequences for our planet and its inhabitants. Rising global temperatures are melting polar ice caps, sea levels are rising, and extreme weather events are becoming more frequent and intense. The consequences of inaction are dire, with devastating impacts on ecosystems, biodiversity, and human societies. However, there is hope. By transitioning to renewable energy sources, increasing energy efficiency, and adopting sustainable land-use practices, we can mitigate the effects of climate change and create a more sustainable future. It's essential that we take immediate action to reduce our carbon footprint and work together to combat this global crisis.

Would you like me to assist with anything else?`
Observation: Invalid or incomplete response
Thought:Thought: Do I

In [20]:
### Example interaction-3
query = "Write a paragraph on importance of sports"
prmpt = build_paragraph_writer_prompt(query)
response = agent.run(query)
print(f"Q: {query}\nResponse:\n{response}\n")

query = f"What is the word count of this paragraph: {response}?"
response = agent.run(query)
print(f"Q: {query}\nResponse:\n{response}")



> Entering new AgentExecutor chain...
Here is the response:

Thought: Do I need to use a tool? No
AI: Sports play a vital role in our society, promoting physical and mental well-being, social bonding, and character development. Participating in sports can improve our overall health by reducing the risk of chronic diseases, enhancing mental health by reducing stress and anxiety, and increasing self-esteem by fostering a sense of accomplishment. Moreover, sports provide a platform for social interaction, teamwork, and communication, helping to build strong relationships and a sense of community. Additionally, sports can serve as a powerful tool for personal development, teaching valuable life skills such as perseverance, discipline, and resilience.

Please let me know if you'd like me to assist with anything else!

> Finished chain.
Q: Write a paragraph on importance of sports
Response:
Sports play a vital role in our society, promoting physical and mental well-being, social bonding, a

In [21]:
### Example interaction-4
query = "Write a paragraph on technology in education"
prmpt = build_paragraph_writer_prompt(query)
response = agent.run(query)
print(f"Q: {query}\nResponse:\n{response}\n")

query = f"What is the word count of this paragraph: {response}?"
response = agent.run(query)
print(f"Q: {query}\nResponse:\n{response}")



> Entering new AgentExecutor chain...
Here is the response:

AI: Technology has revolutionized education by providing digital tools that enhance learning experiences, offer flexibility, and increase accessibility for students around the globe. With the use of technology, students can now access a vast array of educational resources, including online courses, educational apps, and digital textbooks. This has enabled students to learn at their own pace, anytime and anywhere, and has also helped to bridge the gap between students who have access to quality education and those who do not. Additionally, technology has enabled educators to create more engaging and interactive learning experiences, such as virtual labs and simulations, that can help to increase student motivation and participation.

Please let me know if you'd like me to assist with anything else!

Word count: 97 words.

Would you like me to generate a new paragraph or assist with a different task?

> Finished chain.
Q: Wri